# FaceNet (InceptionResnetV1) + ArcFace Training

This notebook trains a FaceNet model using the `facenet-pytorch` library with an ArcFace head on the 105_classes_pins_dataset.

## Key Features:
- **Backbone**: InceptionResnetV1 (pretrained on VGGFace2)
- **Loss**: ArcFace (Additive Angular Margin Loss)
- **Optimizer**: SGD with OneCycleLR scheduler
- **Image Size**: 160x160 (Native resolution for FaceNet)

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## 1. Install Dependencies

In [2]:
!pip install --upgrade torchvision
!pip install facenet-pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 62.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 120.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/

## 2. Imports & Configuration

In [5]:
cd /content/drive/MyDrive/famous Faces projects/deep learning

/content/drive/MyDrive/famous Faces projects/deep learning


In [2]:

!pip install facenet-pytorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 88.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torch.cuda.amp import autocast, GradScaler
from torchvision import datasets, transforms,models
from torchvision.models import VGG16_Weights

from tqdm.notebook import tqdm
import os
import math
import matplotlib.pyplot as plt
import numpy as np

# Configuration
DATASET_PATH = "/content/drive/MyDrive/famous Faces projects/deep learning/105_classes_pins_dataset"
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 64
IMAGE_SIZE = 160  # FaceNet native size
EMBEDDING_SIZE = 512
NUM_CLASSES = 105
EPOCHS = 25

print(f"Using device: {DEVICE}")

Using device: cuda:0


## 3. Data Loading

In [7]:
class TransformSubset(torch.utils.data.Dataset):
    def __init__(self, subset, transform=None):
        self.subset = subset
        self.transform = transform

    def __getitem__(self, index):
        x, y = self.subset[index]
        if self.transform:
            x = self.transform(x)
        return x, y

    def __len__(self):
        return len(self.subset)

# Transforms - Using 160x160 for FaceNet
traintransform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]) # FaceNet expects [-1, 1] usually, or standard normalization.
    # Note: facenet-pytorch's fixed_image_standardization is approx mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]
])

valtransform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Load Dataset
if os.path.exists(DATASET_PATH):
    full_dataset = datasets.ImageFolder(DATASET_PATH)
    train_size = int(0.8 * len(full_dataset))
    val_size = len(full_dataset) - train_size

    train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

    train_dataset = TransformSubset(train_dataset, traintransform)
    val_dataset = TransformSubset(val_dataset, valtransform)

    train_loader = DataLoader(
        train_dataset, batch_size=BATCH_SIZE, shuffle=True,
        num_workers=4, pin_memory=True, persistent_workers=True
    )
    val_loader = DataLoader(
        val_dataset, batch_size=BATCH_SIZE, shuffle=False,
        num_workers=4, pin_memory=True
    )

    print(f"Classes: {len(full_dataset.classes)}")
    print(f"Train: {len(train_dataset)} | Val: {len(val_dataset)}")
else:
    print("WARNING: Dataset path not found. Please check the path.")

Classes: 105
Train: 14027 | Val: 3507


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


## 4. Model Architecture

We use `InceptionResnetV1` as the backbone and attach an `ArcFaceMarginProduct` head.

In [16]:
class ArcFaceMarginProduct(nn.Module):
    """ArcFace margin product"""
    def __init__(self, embedding_size, n_classes, s=30.0, m=0.5, easy_margin=False):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(n_classes, embedding_size))
        nn.init.xavier_uniform_(self.weight)
        self.s = s
        self.m = m
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m
        self.easy_margin = easy_margin

    def forward(self, embeddings, labels=None):
        W = F.normalize(self.weight, dim=1)
        cos_theta = torch.mm(embeddings, W.t()).clamp(-1.0, 1.0)

        if labels is None:
            return cos_theta * self.s

        cos_y = cos_theta.gather(1, labels.view(-1, 1)).view(-1)
        sin_theta = torch.sqrt(1.0 - cos_y * cos_y).clamp(0.0, 1.0)
        phi = cos_y * self.cos_m - sin_theta * self.sin_m

        if self.easy_margin:
            phi = torch.where(cos_y > 0, phi, cos_y)
        else:
            phi = torch.where(cos_y > self.th, phi, cos_y - self.mm)

        logits = cos_theta.clone()
        logits.scatter_(1, labels.view(-1, 1), phi.view(-1, 1))
        logits = logits * self.s
        return logits

class FaceNetArcFace(nn.Module):
    def __init__(self, n_classes, embedding_size=512, pretrained='vggface2', s=30.0, m=0.5):
        super().__init__()
        # Load pretrained FaceNet (InceptionResnetV1)
        self.backbone = InceptionResnetV1(pretrained=pretrained, classify=False)

        # ArcFace Head
        self.margin_product = ArcFaceMarginProduct(
            embedding_size=embedding_size,
            n_classes=n_classes,
            s=s,
            m=m
         )
#vgg model



    def forward(self, x, labels=None):
        # Get embeddings from backbone
        embeddings = self.backbone(x)

        # InceptionResnetV1 output is already normalized if classify=False?
        # Actually, facenet-pytorch implementation does NOT normalize by default if classify=False.
        # It returns x which is the output of the last BN layer.
        # So we should normalize it for ArcFace.
        embeddings = F.normalize(embeddings, dim=1)

        logits = self.margin_product(embeddings, labels)
        return logits, embeddings

## 5. Training Functions

In [20]:
def train_one_epoch(model, train_loader, optimizer, criterion, scaler, scheduler, device, epoch):
    model.train()
    running_loss = 0
    correct = 0
    total = 0

    loop = tqdm(train_loader, desc=f"Epoch {epoch} [TRAIN]")

    for imgs, labels in loop:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()

        with autocast():
            logits, _ = model(imgs, labels)
            loss = criterion(logits, labels)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        scaler.step(optimizer)
        scaler.update()
        scheduler.step()

        running_loss += loss.item() * imgs.size(0)
        preds = logits.argmax(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

        loop.set_postfix(loss=loss.item(), acc=correct/total)

    return running_loss / total, correct / total

def validate_one_epoch(model, val_loader, criterion, device, epoch):
    model.eval()
    running_loss = 0
    correct = 0
    total = 0

    loop = tqdm(val_loader, desc=f"Epoch {epoch} [VAL]")

    with torch.no_grad():
        for imgs, labels in loop:
            imgs, labels = imgs.to(device), labels.to(device)
            with autocast():
                logits, _ = model(imgs, labels)
                loss = criterion(logits, labels)

            running_loss += loss.item() * imgs.size(0)
            preds = logits.argmax(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)
            loop.set_postfix(loss=loss.item(), acc=correct/total)

    return running_loss / total, correct / total

## 6. Main Execution

In [9]:
# Initialize Model
model = FaceNetArcFace(
    n_classes=NUM_CLASSES,
    embedding_size=EMBEDDING_SIZE,
    pretrained='vggface2',
    s=30.0,
    m=0.5
).to(DEVICE)

criterion = nn.CrossEntropyLoss()

# Optimizer - Different LR for backbone and head
optimizer = torch.optim.SGD([
    {'params': model.backbone.parameters(), 'lr': 0.001},  # Lower LR for pretrained backbone
    {'params': model.margin_product.parameters(), 'lr': 0.01} # Higher LR for new head
], momentum=0.9, weight_decay=5e-4)

# Scheduler
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer,
    max_lr=[0.001, 0.01],
    epochs=EPOCHS,
    steps_per_epoch=len(train_loader),
    pct_start=0.2,
    div_factor=10.0,
    final_div_factor=100.0
)

scaler = GradScaler()

# Training Loop
best_val_acc = 0.0
history = {'train_loss': [], 'train_acc': [], 'val_loss': [], 'val_acc': []}

print("Starting training...")
for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train_one_epoch(
        model, train_loader, optimizer, criterion, scaler, scheduler, DEVICE, epoch
    )
    val_loss, val_acc = validate_one_epoch(
        model, val_loader, criterion, DEVICE, epoch
    )

    history['train_loss'].append(train_loss)
    history['train_acc'].append(train_acc)
    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)

    print(f"Epoch {epoch}/{EPOCHS}:")
    print(f"  Train Loss: {train_loss:.4f}, Acc: {train_acc:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Acc: {val_acc:.4f}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "/kaggle/working/best_facenet_arcface.pth")
        print(f"  🔥 New best model saved! ({best_val_acc:.4f})")

# Plotting
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.title('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history['train_acc'], label='Train Acc')
plt.plot(history['val_acc'], label='Val Acc')
plt.title('Accuracy')
plt.legend()
plt.show()

  0%|          | 0.00/107M [00:00<?, ?B/s]

Starting training...


/usr/local/lib/python3.12/dist-packages/torch/cuda/amp/grad_scaler.py:126: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(


Epoch 1 [TRAIN]:   0%|          | 0/220 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(


KeyboardInterrupt: 